In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import os 
import pandas as pd
import requests 
import json
import time 
import scipy.stats as st
from citipy import citipy
import datetime

#National Park Service API 

API_Key_Park = 'x4r30IhC7l6xEe3DCmQmuiefih8dcjFcPutVzRhh'
base_url = "https://api.nps.gov/api/v1/parks?stateCode="

States = ['TX','NM','AZ','OK','LA','AR','WV','KY','TN','NC','SC','GA','AL','MS','FL']

In [2]:
URL = 'https://irmaservices.nps.gov/v2/rest/unit/?unitCodes={unitcode}&format=json'
response = requests.get(URL).json()

column_names = ['FullName','UnitCode','UnitName',
                'StateCode']

state_Codes = []

national_parks = []
for park in response:
    if(park['UnitDesignationName'] == 'National Park'):
        national_parks.append([park['FullName'], 
                          park['UnitCode'], 
                          park['UnitName'],
                          park['StateCodes']])
        state_Codes.append(park['StateCodes'])
    
national_parks
np_df = pd.DataFrame(national_parks, columns=column_names)
np_df.head()

,FullName,UnitCode,UnitName,StateCode
0,Acadia National Park,ACAD,Acadia,[ME]
1,Arches National Park,ARCH,Arches,[UT]
2,Badlands National Park,BADL,Badlands,[SD]
3,Big Bend National Park,BIBE,Big Bend,[TX]
4,Biscayne National Park,BISC,Biscayne,[FL]


In [3]:
column_names = ['UnitCode','Month', 'Visitors']
query_year = "2019"
park_visitor_data = []

for unitcode in np_df['UnitCode']:
    URL = f'https://irmaservices.nps.gov/v3/rest/stats/visitation?unitCodes={unitcode}&startMonth=01&startYear={query_year}&endMonth=12&endYear={query_year}&format=json'
    response = requests.get(URL).json()
    
    for month in range(0,len(response)):
        park_visitor_data.append([unitcode, 
                          month,
                          response[month]['RecreationVisitors']])
                   
visitor_df = pd.DataFrame(park_visitor_data, columns=column_names)
visitor_df1 = visitor_df.groupby('UnitCode')
visitor_df_group = visitor_df1.sum()


,Month,Visitors
UnitCode,,
ACAD,66,3437286
ARCH,66,1659702
BADL,66,970998
BIBE,66,463832
BISC,66,708522


In [4]:
Unique_Unit_Codes = visitor_df['UnitCode'].unique()
codes = np_df['UnitCode']
statec = []

unitcodes = visitor_df['UnitCode']

for i in range(len(Unique_Unit_Codes)):
    for j in range(len(codes)):
        if Unique_Unit_Codes[i] == codes[j]:
            statec.append(state_Codes[j])

In [5]:
VISITORS = visitor_df_group.reset_index()

In [6]:
VISITORS['State'] = statec

In [7]:
south = []
for i in range(len(VISITORS)):
    for state in States:
        if VISITORS['State'][i] == [f'{state}']:
            south.append([VISITORS['UnitCode'][i],
                          VISITORS['Visitors'][i],
                          VISITORS['State'][i]])
            
south_df = pd.DataFrame(south, columns = ['Park Code','Visitors','State'])

In [8]:
south_df

,Park Code,Visitors,State
0,BIBE,463832,[TX]
1,BISC,708522,[FL]
2,CAVE,440691,[NM]
3,CONG,159445,[SC]
4,DRTO,79200,[FL]
5,EVER,1118300,[FL]
6,GRCA,5974411,[AZ]
7,GUMO,188833,[TX]
8,HOSP,1467153,[AR]
9,MACA,551590,[KY]
